# 第二章：使用文本数据

本章节中中使用的软件包：

In [ ]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))
print("transformers version:", version("transformers"))

torch version: 2.5.1
tiktoken version: 0.8.0
transformers version: 4.46.3


## 2.1 理解词的嵌入

- 嵌入有多种形式，例如视频嵌入、语音嵌入、文本嵌入；本书重点介绍文本嵌入
- LLM适用于高维空间（即数千个维度）中的嵌入
- 由于我们人类无法想象这样的高维空间（我们人类以1、2或3维思考）

## 2.2 文本标记

- 将文本进行标记就是将文本分解为更小的单元，例如单个词和标点符号

- 加载我们要处理的原始文本
- [The Verdict](https://github.com/GavinHome/LLMs-from-scratch-zh/blob/main/ch02/the-verdict.txt) 是一个公开的短篇小说的中文译文版本

In [2]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/GavinHome/"
           "LLMs-from-scratch-zh/main/ch02/the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

- 读取原始文本

In [12]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("总字数:", len(raw_text))
print(raw_text[:50])

总字数: 2806
我一直认为杰克·吉斯伯恩是一个廉价的天才——尽管他是个不错的家伙——所以当我听说他在事业巅峰时放弃了


- 目标是为LLM标记并嵌入此文本
- 首先进行分词，有很多方法，我们选择transformers库中的"bert-base-chinese"来进行中文分词,将这个过程称为Tokenized

In [77]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")

tokens = tokenizer.tokenize(raw_text)
print(tokens[:10])

['我', '一', '直', '认', '为', '杰', '克', '·', '吉', '斯']


- 让我们计算一下 tokens 的总数

In [170]:
print(len(tokens))

2756


## 2.3 将tokens转化为 token IDs

- 接下来，我们将文本 tokens 转化为 token IDs，以便稍后通过嵌入层处理
- 简单的方法就是将转化后的 tokens 中所有 token 进行编号，然后增加一些特殊的符号的编号（开始标记，结束标记，分割标记等），最后将上述文本中每个 token按照编号对应，最后形成对应 tokenID 集合
- 但上述方法过于简单，需要考虑很多特殊情况，索性 transformers 库中有专门用于处理这种问题的函数，我们可以直接使用

In [171]:
print(tokenizer.convert_tokens_to_ids(tokens))

[2769, 671, 4684, 6371, 711, 3345, 1046, 185, 1395, 3172, 843, 2617, 3221, 671, 702, 2442, 817, 4638, 1921, 2798, 100, 100, 2226, 5052, 800, 3221, 702, 679, 7231, 4638, 2157, 832, 100, 100, 2792, 809, 2496, 2769, 1420, 6432, 800, 1762, 752, 689, 2330, 2292, 3198, 3123, 2461, 749, 5313, 4514, 8024, 2034, 749, 671, 855, 2168, 3300, 4638, 2176, 1967, 8024, 2400, 1762, 7027, 5335, 1812, 2861, 4638, 671, 2429, 1166, 1863, 7027, 2128, 7561, 678, 3341, 3198, 8024, 2769, 2400, 679, 2697, 1168, 4294, 1166, 2661, 6385, 511, 8020, 6006, 4197, 2769, 3291, 967, 1403, 754, 6371, 711, 800, 833, 6848, 2885, 5384, 7716, 2772, 867, 5384, 840, 5855, 511, 8021, 100, 752, 689, 4638, 2330, 2292, 100, 100, 100, 6929, 3221, 1957, 1894, 812, 2190, 800, 4638, 4917, 1461, 511, 2769, 5543, 1420, 1168, 1395, 6832, 2617, 185, 3172, 3946, 1922, 1922, 100, 100, 800, 3297, 1400, 671, 855, 5698, 1217, 1520, 4638, 3563, 4294, 1036, 100, 100, 2190, 800, 679, 1377, 4415, 6237, 4638, 6842, 7391, 6134, 4850, 6890, 2742, 511

## 2.4 添加特殊 tokens

- 为未知词添加一些“特殊”标记并表示文本的结束很有用
- 一些标记器使用特殊标记来帮助 LLM 提供额外的上下文
- 其中一些特殊标记是
  - `[BOS]`（序列开头）标记文本的开头
  - `[EOS]`（序列结尾）标记文本的结尾（这通常用于连接多个不相关的文本，例如，两个不同的维基百科文章或两本不同的书）
  - `[PAD]`（填充）如果我们训练批量大小大于 1 的 LLM（可能包含多个不同长度的文本；使用填充标记，将较短的文本填充到最长的长度，以便所有文本的长度相等）
  - `[UNK]` 表示未包含在词汇表中的单词
- 索性这些特殊标记的处理，transformers 库有相应的方法进行了处理，我们只需要直接使用即可

## 2.5 字节对编码

In [187]:
integers = tokenizer.encode(raw_text)
print(integers)

[101, 2769, 671, 4684, 6371, 711, 3345, 1046, 185, 1395, 3172, 843, 2617, 3221, 671, 702, 2442, 817, 4638, 1921, 2798, 100, 100, 2226, 5052, 800, 3221, 702, 679, 7231, 4638, 2157, 832, 100, 100, 2792, 809, 2496, 2769, 1420, 6432, 800, 1762, 752, 689, 2330, 2292, 3198, 3123, 2461, 749, 5313, 4514, 8024, 2034, 749, 671, 855, 2168, 3300, 4638, 2176, 1967, 8024, 2400, 1762, 7027, 5335, 1812, 2861, 4638, 671, 2429, 1166, 1863, 7027, 2128, 7561, 678, 3341, 3198, 8024, 2769, 2400, 679, 2697, 1168, 4294, 1166, 2661, 6385, 511, 8020, 6006, 4197, 2769, 3291, 967, 1403, 754, 6371, 711, 800, 833, 6848, 2885, 5384, 7716, 2772, 867, 5384, 840, 5855, 511, 8021, 100, 752, 689, 4638, 2330, 2292, 100, 100, 100, 6929, 3221, 1957, 1894, 812, 2190, 800, 4638, 4917, 1461, 511, 2769, 5543, 1420, 1168, 1395, 6832, 2617, 185, 3172, 3946, 1922, 1922, 100, 100, 800, 3297, 1400, 671, 855, 5698, 1217, 1520, 4638, 3563, 4294, 1036, 100, 100, 2190, 800, 679, 1377, 4415, 6237, 4638, 6842, 7391, 6134, 4850, 6890, 2742

In [189]:
strings = tokenizer.decode(integers)

print(strings)

[CLS] 我 一 直 认 为 杰 克 · 吉 斯 伯 恩 是 一 个 廉 价 的 天 才 [UNK] [UNK] 尽 管 他 是 个 不 错 的 家 伙 [UNK] [UNK] 所 以 当 我 听 说 他 在 事 业 巅 峰 时 放 弃 了 绘 画 ， 娶 了 一 位 富 有 的 寡 妇 ， 并 在 里 维 埃 拉 的 一 座 别 墅 里 安 顿 下 来 时 ， 我 并 不 感 到 特 别 惊 讶 。 （ 虽 然 我 更 倾 向 于 认 为 他 会 选 择 罗 马 或 佛 罗 伦 萨 。 ） [UNK] 事 业 的 巅 峰 [UNK] [UNK] [UNK] 那 是 女 士 们 对 他 的 称 呼 。 我 能 听 到 吉 迪 恩 · 斯 温 太 太 [UNK] [UNK] 他 最 后 一 位 芝 加 哥 的 模 特 儿 [UNK] [UNK] 对 他 不 可 理 解 的 退 隐 表 示 遗 憾 。 [UNK] 当 然 ， 这 会 让 我 那 幅 画 的 价 值 飙 升 ； 但 我 并 不 这 么 想 ， 里 克 汉 先 生 [UNK] [UNK] 我 只 关 心 艺 术 的 损 失 。 [UNK] 这 个 词 在 斯 温 太 太 口 中 被 重 复 了 多 次 ， 仿 佛 它 们 在 无 尽 的 镜 像 中 反 射 。 不 仅 斯 温 太 太 们 哀 悼 ， 难 道 不 是 吗 ？ 在 最 后 一 次 格 拉 夫 顿 画 廊 展 览 上 ， 精 致 的 赫 米 娅 · 克 罗 夫 特 在 吉 斯 伯 恩 的 《 月 舞 者 》 前 停 下 脚 步 ， 眼 里 含 着 泪 水 对 我 说 ： [UNK] 我 们 再 也 看 不 到 这 样 的 作 品 了 [UNK] ？ 好 吧 ！ 即 使 透 过 赫 米 娅 的 眼 泪 ， 我 也 能 平 静 地 面 对 这 个 事 实 。 可 怜 的 杰 克 · 吉 斯 伯 恩 ！ 女 人 们 造 就 了 他 [UNK] [UNK] 她 们 哀 悼 他 是 理 所 当 然 的 。 在 他 自 己 的 性 别 中 ， 很 少 有 人 表 示 遗 憾 ， 在 他 的 行 业 中 几 乎 听 不 到 任 何 声 音 。 职 业 嫉 妒 ？ 也 许 吧 。 如 果 真 是 这 样 ， 那 么 小 克 劳 德 · 纳 特 利 的 行 为 

- [CLS] 通常位于每个输入序列的开头，代表整个序列的聚合表示
- [SEP] 通常用于分隔不同句子或段落

## 2.6 使用滑动窗口进行数据采样

- 我们训练LLM每次生成一个字或词，因此我们希望相应地准备训练数据，其中序列的下一个字或词代表要预测的目标：

In [190]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
enc_text = tokenizer.encode(raw_text, max_length=4096, truncation=True)
print(len(enc_text))

2758


- 对于每个文本块，我们需要输入和输出
- 因为我们希望模型预测下一个字或词，所以输出是向右移动一个位置的输入

In [179]:
enc_sample = enc_text[:]
print(enc_sample)

[101, 2769, 671, 4684, 6371, 711, 3345, 1046, 185, 1395, 3172, 843, 2617, 3221, 671, 702, 2442, 817, 4638, 1921, 2798, 100, 100, 2226, 5052, 800, 3221, 702, 679, 7231, 4638, 2157, 832, 100, 100, 2792, 809, 2496, 2769, 1420, 6432, 800, 1762, 752, 689, 2330, 2292, 3198, 3123, 2461, 749, 5313, 4514, 8024, 2034, 749, 671, 855, 2168, 3300, 4638, 2176, 1967, 8024, 2400, 1762, 7027, 5335, 1812, 2861, 4638, 671, 2429, 1166, 1863, 7027, 2128, 7561, 678, 3341, 3198, 8024, 2769, 2400, 679, 2697, 1168, 4294, 1166, 2661, 6385, 511, 8020, 6006, 4197, 2769, 3291, 967, 1403, 754, 6371, 711, 800, 833, 6848, 2885, 5384, 7716, 2772, 867, 5384, 840, 5855, 511, 8021, 100, 752, 689, 4638, 2330, 2292, 100, 100, 100, 6929, 3221, 1957, 1894, 812, 2190, 800, 4638, 4917, 1461, 511, 2769, 5543, 1420, 1168, 1395, 6832, 2617, 185, 3172, 3946, 1922, 1922, 100, 100, 800, 3297, 1400, 671, 855, 5698, 1217, 1520, 4638, 3563, 4294, 1036, 100, 100, 2190, 800, 679, 1377, 4415, 6237, 4638, 6842, 7391, 6134, 4850, 6890, 2742

In [180]:
context_size = 8

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [101, 2769, 671, 4684, 6371, 711, 3345, 1046]
y:      [2769, 671, 4684, 6371, 711, 3345, 1046, 185]


- 逐一进行预测结果如下：

In [181]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

[101] ----> 2769
[CLS] ----> 我
[101, 2769] ----> 671
[CLS] 我 ----> 一
[101, 2769, 671] ----> 4684
[CLS] 我 一 ----> 直
[101, 2769, 671, 4684] ----> 6371
[CLS] 我 一 直 ----> 认
[101, 2769, 671, 4684, 6371] ----> 711
[CLS] 我 一 直 认 ----> 为
[101, 2769, 671, 4684, 6371, 711] ----> 3345
[CLS] 我 一 直 认 为 ----> 杰
[101, 2769, 671, 4684, 6371, 711, 3345] ----> 1046
[CLS] 我 一 直 认 为 杰 ----> 克
[101, 2769, 671, 4684, 6371, 711, 3345, 1046] ----> 185
[CLS] 我 一 直 认 为 杰 克 ----> ·


- 为了实现下一个字或词的预测，首先需要实现一个数据加载器，它遍历输入数据集并返回移位一个的输入和输出

In [178]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.5.1+cu124


- 创建数据集和数据加载器，从输入文本数据集中提取文本块，们使用滑动窗口方法，将位置改变 +1

In [200]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, max_length=None, truncation=False)

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [192]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

- 让我们针对上下文大小为 8 的 LLM， 测试批量大小为 1 的数据加载器：

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=8, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)
x1, y1 = first_batch
print("第一批: ", tokenizer.decode(x1.squeeze()), "---->", tokenizer.decode(y1.squeeze()))

Token indices sequence length is longer than the specified maximum sequence length for this model (2758 > 512). Running this sequence through the model will result in indexing errors


[tensor([[ 101, 2769,  671, 4684, 6371,  711, 3345, 1046]]), tensor([[2769,  671, 4684, 6371,  711, 3345, 1046,  185]])]
第一批:  [CLS] 我 一 直 认 为 杰 克 ----> 我 一 直 认 为 杰 克 ·


In [195]:
second_batch = next(data_iter)
print(second_batch)
x2, y2 = second_batch
print("第二批: ", tokenizer.decode(x2.squeeze()), "---->", tokenizer.decode(y2.squeeze()))

[tensor([[2769,  671, 4684, 6371,  711, 3345, 1046,  185]]), tensor([[ 671, 4684, 6371,  711, 3345, 1046,  185, 1395]])]
第二批:  我 一 直 认 为 杰 克 · ----> 一 直 认 为 杰 克 · 吉


- 我们还可以创建分批输出
- 请注意，我们在这里增加了步幅，这样批次之间就不会出现重叠，因为更多的重叠可能会导致过度拟合

In [204]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=8, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("输入: ", list(map(lambda batch: tokenizer.decode(batch, tokenizer), inputs.tolist())))
print("\nTargets:\n", targets)
print("输出: ", list(map(lambda batch: tokenizer.decode(batch, tokenizer), targets.tolist())))

Token indices sequence length is longer than the specified maximum sequence length for this model (2758 > 512). Running this sequence through the model will result in indexing errors


Inputs:
 tensor([[ 101, 2769,  671, 4684, 6371,  711, 3345, 1046],
        [6371,  711, 3345, 1046,  185, 1395, 3172,  843],
        [ 185, 1395, 3172,  843, 2617, 3221,  671,  702],
        [2617, 3221,  671,  702, 2442,  817, 4638, 1921],
        [2442,  817, 4638, 1921, 2798,  100,  100, 2226],
        [2798,  100,  100, 2226, 5052,  800, 3221,  702],
        [5052,  800, 3221,  702,  679, 7231, 4638, 2157],
        [ 679, 7231, 4638, 2157,  832,  100,  100, 2792]])
输入:  ['我 一 直 认 为 杰 克', '认 为 杰 克 · 吉 斯 伯', '· 吉 斯 伯 恩 是 一 个', '恩 是 一 个 廉 价 的 天', '廉 价 的 天 才 尽', '才 尽 管 他 是 个', '管 他 是 个 不 错 的 家', '不 错 的 家 伙 所']

Targets:
 tensor([[2769,  671, 4684, 6371,  711, 3345, 1046,  185],
        [ 711, 3345, 1046,  185, 1395, 3172,  843, 2617],
        [1395, 3172,  843, 2617, 3221,  671,  702, 2442],
        [3221,  671,  702, 2442,  817, 4638, 1921, 2798],
        [ 817, 4638, 1921, 2798,  100,  100, 2226, 5052],
        [ 100,  100, 2226, 5052,  800, 3221,  702,  679],
        [ 800, 3221,  7

In [203]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=8, stride=4, shuffle=False)

print("Train loader:")
for x, y in dataloader:
    print(x.shape, y.shape)

Token indices sequence length is longer than the specified maximum sequence length for this model (2758 > 512). Running this sequence through the model will result in indexing errors


Train loader:
torch.Size([8, 8]) torch.Size([8, 8])
torch.Size([8, 8]) torch.Size([8, 8])
torch.Size([8, 8]) torch.Size([8, 8])
torch.Size([8, 8]) torch.Size([8, 8])
torch.Size([8, 8]) torch.Size([8, 8])
torch.Size([8, 8]) torch.Size([8, 8])
torch.Size([8, 8]) torch.Size([8, 8])
torch.Size([8, 8]) torch.Size([8, 8])
torch.Size([8, 8]) torch.Size([8, 8])
torch.Size([8, 8]) torch.Size([8, 8])
torch.Size([8, 8]) torch.Size([8, 8])
torch.Size([8, 8]) torch.Size([8, 8])
torch.Size([8, 8]) torch.Size([8, 8])
torch.Size([8, 8]) torch.Size([8, 8])
torch.Size([8, 8]) torch.Size([8, 8])
torch.Size([8, 8]) torch.Size([8, 8])
torch.Size([8, 8]) torch.Size([8, 8])
torch.Size([8, 8]) torch.Size([8, 8])
torch.Size([8, 8]) torch.Size([8, 8])
torch.Size([8, 8]) torch.Size([8, 8])
torch.Size([8, 8]) torch.Size([8, 8])
torch.Size([8, 8]) torch.Size([8, 8])
torch.Size([8, 8]) torch.Size([8, 8])
torch.Size([8, 8]) torch.Size([8, 8])
torch.Size([8, 8]) torch.Size([8, 8])
torch.Size([8, 8]) torch.Size([8, 8]

## 2.7 创建标记嵌入

- 数据几乎已准备好用于 LLM
- 但最后让我们使用嵌入层将标记嵌入到连续向量表示中
- 通常，这些嵌入层是 LLM 本身的一部分，并在模型训练期间进行更新（训练）

- 假设我们有以下四个输入示例，输入 ID 分别为 2、3、5 和 1（标记后）：

In [206]:
input_ids = torch.tensor([2, 3, 5, 1])

- 为了简单起见，假设我们的词汇表只有 6 个词，并且我们想要创建大小为 3 的嵌入：

In [216]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- 这将产生一个 6x3 的权重矩阵：

In [217]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


- 要将 id 为 3 的 token 转换为三维向量，我们执行以下操作：

In [218]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


- 请注意，上面是 `embedding_layer` 权重矩阵中的第 4 行
- 要嵌入上面所有四个 `input_ids` 值，我们这样做

In [219]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


- 嵌入层本质上是一种查找操作：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/16.webp?123" width="500px">

## 2.8 编码单词位置

- 嵌入层将 ID 转换为相同的向量表示，无论它们位于输入序列的什么位置
- 位置嵌入与标记嵌入向量相结合，形成大型语言模型的输入嵌入

- BytePair 编码器的词汇量为 len(tokenizer):
- 假设我们要将输入标记编码为 256 维向量表示：

In [ ]:
vocab_size = len(tokenizer)
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- 如果我们从数据加载器中抽样数据，我们会将每批中的标记嵌入到 256 维向量中
- 如果批大小为 8，每批有 4 个标记，则会产生 8 x 4 x 256 张量：

In [233]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

Token indices sequence length is longer than the specified maximum sequence length for this model (2758 > 512). Running this sequence through the model will result in indexing errors


In [234]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


- 使用绝对位置嵌入，因此我们只需创建另一个嵌入层：

In [235]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [236]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


- 要创建 LLM 中使用的输入嵌入，我们只需添加标记和位置嵌入：

In [237]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])


- 在输入处理工作流程的初始阶段，输入文本被分割成单独的标记
- 分割之后，这些标记将根据预定义的词汇转换为标记 ID
- 然后经过标记嵌入和位置嵌入，最后形成输入嵌入层